<a id="toc"></a>

# <u>Table of Contents</u>
1.) [TODO](#todo)  
2.) [Imports](#imports)  
3.) [Load data](#load)  
4.) [Bureau](#bureau)  
&nbsp;&nbsp;&nbsp;&nbsp; 4.1.) [Missing values](#bureau_nan)    
5.) [Previous Application](#prev_app)  
&nbsp;&nbsp;&nbsp;&nbsp; 5.1.) [Missing values](#prev_nan)  
6.) [Bureau Balance](#bureau_bal)  
&nbsp;&nbsp;&nbsp;&nbsp; 6.1.) [Merge into Bureau](#merge_bureau_bal)  
7.) [POS CASH balance](#pos_cash)  
&nbsp;&nbsp;&nbsp;&nbsp; 7.1.) [Missing values](#pos_nan)  
&nbsp;&nbsp;&nbsp;&nbsp; 7.2.) [Merge into Previous Application](#merge_pos_cash)  
8.) [Installment Payments](#install_pay)  
&nbsp;&nbsp;&nbsp;&nbsp; 8.1.) [Missing values](#install_nan)  
&nbsp;&nbsp;&nbsp;&nbsp; 8.2.) [Merge into Previous Application](#merge_install_pay)  
9.) [Credit Card Balance](#credit)  
&nbsp;&nbsp;&nbsp;&nbsp; 9.1.) [Missing values](#credit_nan)  
&nbsp;&nbsp;&nbsp;&nbsp; 9.2.) [Merge into Previous Application](#merge_credit)  
10.) [Misc clean up](#clean_up)  
11.) [Final Data Prep](#final_merge)  
&nbsp;&nbsp;&nbsp;&nbsp; 11.1.) [Missing values](#final_nan)  
12.) [Modeling](#models)  
13.) [Predictions](#predictions)  
14.) [Save file to CSV](#save)  

<a id="todo"></a>

# [^](#toc) <u>TODO</u>

- Fix skew on columns (prev_app -> AMT_ANNUITY + AMT_APPLICATION)
- Tinker with the best way to replace missing values (dropping cols?)
- Look for outliers
- Include timeline relatoinships like MONTHS_BALANCE
- Tune model parameters
- Address [this](https://www.kaggle.com/c/home-credit-default-risk/discussion/57248)

---
<a id="imports"></a>

# [^](#toc) <u>Imports</u>

In [1]:
### Standard imports
import pandas as pd
import numpy as np

# Time keeper
import time

# Progress bar
from tqdm import tqdm

# Modeling imports
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

### Removes warnings from output
import warnings
warnings.filterwarnings('ignore')

### Helper functions

In [2]:
# function to create dummy variables of categorical features
def get_dummies(df, cats):
    for col in cats:
        df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df 

def fillna_num(df):
    missing_cols = [col for col in df.columns if any(df[col].isnull()) and df[col].dtype != object]
    for col in missing_cols:
        df[col] = df[col].fillna(df[col].median())
    return df

def fillna_cat(df):
    for col in [col for col in df if df[col].dtype==object]:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df

def factorize_df(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 

---
<a id="load"></a>

# [^](#toc) <u>Load data</u>

In [3]:
DATA_PATH = "../data/home_default/"

---
<a id="bureau"></a>

# [^](#toc) <u>Bureau</u>

In [4]:
bureau   = pd.read_csv(DATA_PATH + "bureau.csv")

print("Shape of bureau:",    bureau.shape)

Shape of bureau: (1716428, 17)


<a id="bureau_nan"></a>

### [^](#toc) Missing values

In [5]:
cols = ["DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT", "AMT_CREDIT_MAX_OVERDUE",
        "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT",
        "AMT_ANNUITY"]

for col in tqdm(cols):
    bureau[col + "_nan"] = bureau[col].map(lambda x: 1 if np.isnan(x) else 0)
    mode                 = bureau[bureau[col].notnull()][col].mode().iloc[0]
    bureau[col]          = bureau[col].fillna(mode)
    
sum(bureau.isnull().sum())

100%|██████████| 7/7 [00:51<00:00,  7.40s/it]


0

In [6]:
[col for col in bureau.columns if bureau[col].dtype == object]

['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']

###

In [7]:
cols = ['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']

for col in cols:
    temp = bureau[col].value_counts()
    print(temp)

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64
currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64
Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Interbank credit                  

In [ ]:
cat_cols = ['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE', 'SK_ID_CURR']
num_cols = [col for col in bureau.columns if col not in cat_cols]
num_cols.append("SK_ID_CURR")

merge_df      = bureau[num_cols].groupby('SK_ID_CURR').mean()
merge_df["N"] = bureau.groupby('SK_ID_CURR').count().iloc[:,0]

right = bureau[cat_cols].set_index("SK_ID_CURR")
right = pd.get_dummies(merge_df).reset_index()
right = merge_df.groupby("SK_ID_CURR").sum().reset_index()

merge_df = merge_df.reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_CURR").set_index("SK_ID_CURR")

merged_cols   = ['pb_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

full = full.merge(right=merge_df.reset_index(), how='left', on='SK_ID_CURR')

---
<a id="prev_app"></a>

# [^](#toc) <u>Previous Application</u>

In [4]:
prev_app = pd.read_csv(DATA_PATH + "previous_application.csv")

print("Shape of prev_app:",  prev_app.shape)

Shape of prev_app: (1670214, 37)


### [^](#toc) Adjust DAYS variables

According to [this](https://www.kaggle.com/c/home-credit-default-risk/discussion/57248) Kaggle Discussion, we can treat values of 365243 as "NA" values

Note: DAYS_DECISION contains no 365243 values so it isn't included

In [5]:
for col in tqdm(("DAYS_TERMINATION", "DAYS_LAST_DUE", 
                "DAYS_LAST_DUE_1ST_VERSION", "DAYS_FIRST_DUE", "DAYS_FIRST_DRAWING")):
    prev_app[col + "_maxed"] = prev_app[col].map(lambda x: 1 if x == 365243 else 0)
    prev_app[col]            = prev_app[col].map(lambda x: 0 if x == 365243 else x)

100%|██████████| 5/5 [00:08<00:00,  1.68s/it]


<a id="prev_nan"></a>

### [^](#toc) Missing values

In [7]:
### Fillin missing values with filler... NaN
for col in ("NAME_TYPE_SUITE", "PRODUCT_COMBINATION"):
    prev_app[col] = prev_app[col].fillna("NaN")
    
### These values have to little counts, clump as one
misc_vals = ["Hobby", "Money for a third person", "Refusal to name the goal"]
misc_map = lambda x: "Misc" if x in misc_vals else x
prev_app.NAME_CASH_LOAN_PURPOSE = prev_app.NAME_CASH_LOAN_PURPOSE.map(misc_map)

### Drop outliers
prev_app = prev_app.drop(
                            prev_app[(prev_app.NAME_GOODS_CATEGORY == "Animals") |
                                     (prev_app.NAME_GOODS_CATEGORY == "House Construction")].index)

### Light missing values
prev_app["AMT_CREDIT"] = prev_app["AMT_CREDIT"].fillna(prev_app["AMT_CREDIT"].median())

### Moderate missing values
for col in tqdm(("AMT_ANNUITY", "AMT_GOODS_PRICE", "CNT_PAYMENT", "AMT_DOWN_PAYMENT", "RATE_DOWN_PAYMENT",
                "DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", "DAYS_LAST_DUE",
                "DAYS_TERMINATION", "NFLAG_INSURED_ON_APPROVAL")):
    prev_app[col + "_nan"] = prev_app[col].map(lambda x: 1 if np.isnan(x) else 0)
    prev_app[col] = prev_app[col].transform(lambda x: x.fillna(x.median()))
    
### Severe missing values
for col in ("RATE_INTEREST_PRIMARY", "RATE_INTEREST_PRIVILEGED"):
    prev_app[col] = prev_app[col].map(lambda x: 0 if np.isnan(x) else 1)
    
sum(prev_app.isnull().sum())

100%|██████████| 11/11 [00:35<00:00,  3.27s/it]


0

<a id="bureau_bal"></a>

# [^](#toc) <u>Bureau Balance</u>

In [9]:
bureau_balance = pd.read_csv(DATA_PATH + "bureau_balance.csv")
print("Shape of bureau_balance:",  bureau_balance.shape)

print("\nColumns of bureau_balance:")
print(" --- ".join(bureau_balance.columns.values))

Shape of bureau_balance: (27299925, 3)

Columns of bureau_balance:
SK_ID_BUREAU --- MONTHS_BALANCE --- STATUS


<a id="merge_bureau_bal"></a>

### [^](#toc) <u>Merge into Bureau</u>

In [10]:
# Setup bureau balance - get dummies
merge_df = get_dummies(bureau_balance, ["STATUS"])

merge_df = merge_df.drop(["MONTHS_BALANCE", "STATUS"], axis=1)

# prep for merge
merge_df = merge_df.groupby("SK_ID_BUREAU").sum()

### Add the max number of months
merge_df["max_months"] = bureau_balance.groupby("SK_ID_BUREAU")["MONTHS_BALANCE"].max()

### Remember added columns
merged_cols = ['bur_bal_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

# Merge
bureau = bureau.merge(right=merge_df.reset_index(), how='left', on='SK_ID_BUREAU')

### Fill in new missing values

In [11]:
bureau["no_bureau_bal"] = bureau[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)
bureau[merged_cols]     = bureau[merged_cols].fillna(0)
sum(bureau.isnull().sum())

0

---
<a id="pos_cash"></a>

# [^](#toc) <u>POS CASH balance</u>

In [8]:
pcb = pd.read_csv(DATA_PATH + "POS_CASH_balance.csv")
print("Shape of pcb:",  pcb.shape)

print("\nColumns of pcb:")
print(" --- ".join(pcb.columns.values))

Shape of pcb: (10001358, 8)

Columns of pcb:
SK_ID_PREV --- SK_ID_CURR --- MONTHS_BALANCE --- CNT_INSTALMENT --- CNT_INSTALMENT_FUTURE --- NAME_CONTRACT_STATUS --- SK_DPD --- SK_DPD_DEF


<a id="pos_nan"></a>

### [^](#toc) Missing Values

In [9]:
for col in ("CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"):
    pcb[col] = pcb[col].transform(lambda x: x.fillna(x.median()))

### Remove Outliers

pcb.NAME_CONTRACT_STATUS.value_counts()

    Canceled: 15
    XNA:      2

In [10]:
pcb = pcb.drop(pcb[pcb.NAME_CONTRACT_STATUS.isin(["XNA", "Canceled"])].index)

### Get Dummies

In [11]:
merge_df = pcb[["SK_ID_PREV", "NAME_CONTRACT_STATUS"]]

merge_df = get_dummies(merge_df, ["NAME_CONTRACT_STATUS"])
merge_df = merge_df.drop("NAME_CONTRACT_STATUS", axis=1)

<a id="merge_pos_cash"></a>

### [^](#toc) <u>Merge into Previous Application</u>

In [12]:
# prep for merge
count    = merge_df.groupby("SK_ID_PREV").count()
merge_df = merge_df.groupby("SK_ID_PREV").sum().reset_index()
merge_df["N"] = list(count.iloc[:,0])

# Add the median values.  MONTHS_BALANCE will be added as the max
right    = pcb.drop(["SK_ID_CURR", "MONTHS_BALANCE"], axis=1).groupby("SK_ID_PREV").median().reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_PREV").set_index("SK_ID_PREV")

### Add the max number of months
merge_df["max_months"] = pcb.groupby("SK_ID_PREV").MONTHS_BALANCE.max()

merged_cols = ['pos_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

# Merge
prev_app = prev_app.merge(right=merge_df.reset_index(), how='left', on='SK_ID_PREV')

### Fill in missing values

In [13]:
prev_app["no_pcb"] = prev_app[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null      = prev_app[col].notnull()
    mode          = prev_app[not_null][col].mode()
    prev_app[col] = prev_app[col].fillna(mode)    
    
sum(prev_app.isnull().sum())

100%|██████████| 13/13 [00:21<00:00,  1.66s/it]


10027043

---
<a id="install_pay"></a>

# [^](#toc) <u>Installment Payments</u>

In [14]:
install_pay = pd.read_csv(DATA_PATH + "installments_payments.csv")
print("Shape of install_pay:",  install_pay.shape)

print("\nColumns of install_pay:")
print(" --- ".join(install_pay.columns.values))

Shape of install_pay: (13605401, 8)

Columns of install_pay:
SK_ID_PREV --- SK_ID_CURR --- NUM_INSTALMENT_VERSION --- NUM_INSTALMENT_NUMBER --- DAYS_INSTALMENT --- DAYS_ENTRY_PAYMENT --- AMT_INSTALMENT --- AMT_PAYMENT


<a id="install_nan"></a>

### [^](#toc) <u>Missing values</u>

In [15]:
for col in ("DAYS_ENTRY_PAYMENT", "AMT_PAYMENT"):
    install_pay[col + "_nan"] = install_pay[col].map(lambda x: 1 if np.isnan(x) else 0)
    install_pay[col] = install_pay[col].fillna(0)

### Setup for merge

In [16]:
install_pay["AMT_MISSING"] = install_pay["AMT_INSTALMENT"] - install_pay["AMT_PAYMENT"]
temp = install_pay.groupby("SK_ID_PREV")["AMT_MISSING"]

merge_df = pd.DataFrame({
    "INSTALL_missing_max": temp.max(),
    "INSTALL_missing_min": temp.min(),
    "INSTALL_missing_med": temp.median(),
    "INSTALL_payment_nan": install_pay.groupby("SK_ID_PREV")["AMT_PAYMENT_nan"].sum(),
    "INSTALL_N":           temp.count()
})

### Add the rest of the columns

In [17]:
right = install_pay.drop("SK_ID_CURR", axis=1).groupby("SK_ID_PREV").median().reset_index()
merge_df = merge_df.reset_index()

merge_df = merge_df.merge(right=right, how="left", on="SK_ID_PREV").set_index("SK_ID_PREV")
merged_cols = merge_df.columns

<a id="merge_install_pay"></a>

### [^](#toc) <u>Merge into Previous Application</u>

In [18]:
# Merge
prev_app = prev_app.merge(right=merge_df.reset_index(), how='left', on='SK_ID_PREV')

### Fill in missing values

In [19]:
prev_app["no_install"] = prev_app[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null      = prev_app[col].notnull()
    mode          = prev_app[not_null][col].mode()
    prev_app[col] = prev_app[col].fillna(mode)    
    
sum(prev_app.isnull().sum())

100%|██████████| 14/14 [00:32<00:00,  2.33s/it]


19985355

---
<a id="credit"></a>

# [^](#toc) <u>Credit Card Balance</u>

In [20]:
credit_card = pd.read_csv(DATA_PATH + "credit_card_balance.csv")
print("Shape of credit_card:",  credit_card.shape)

print("\nColumns of credit_card:")
print(" --- ".join(credit_card.columns.values))

Shape of credit_card: (3840312, 23)

Columns of credit_card:
SK_ID_PREV --- SK_ID_CURR --- MONTHS_BALANCE --- AMT_BALANCE --- AMT_CREDIT_LIMIT_ACTUAL --- AMT_DRAWINGS_ATM_CURRENT --- AMT_DRAWINGS_CURRENT --- AMT_DRAWINGS_OTHER_CURRENT --- AMT_DRAWINGS_POS_CURRENT --- AMT_INST_MIN_REGULARITY --- AMT_PAYMENT_CURRENT --- AMT_PAYMENT_TOTAL_CURRENT --- AMT_RECEIVABLE_PRINCIPAL --- AMT_RECIVABLE --- AMT_TOTAL_RECEIVABLE --- CNT_DRAWINGS_ATM_CURRENT --- CNT_DRAWINGS_CURRENT --- CNT_DRAWINGS_OTHER_CURRENT --- CNT_DRAWINGS_POS_CURRENT --- CNT_INSTALMENT_MATURE_CUM --- NAME_CONTRACT_STATUS --- SK_DPD --- SK_DPD_DEF


<a id="credit_nan"></a>

### [^](#toc) <u>Missing Values and Outliers</u>

In [21]:
# ------------------------------
### Remove outliers
# Gets indices with outlier values
temp = credit_card[credit_card.NAME_CONTRACT_STATUS.isin(["Refused", "Approved"])].index

# Drops outlier values
credit_card = credit_card.drop(temp, axis=0)

# ------------------------------
#### Fill in missing values
cols = [
        "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT", 
        "AMT_INST_MIN_REGULARITY", "AMT_PAYMENT_CURRENT", "CNT_DRAWINGS_ATM_CURRENT", 
        "CNT_DRAWINGS_OTHER_CURRENT", "CNT_DRAWINGS_POS_CURRENT", "CNT_INSTALMENT_MATURE_CUM"
]
for col in tqdm(cols):
    not_null = credit_card[col].notnull()
    mode = float(credit_card[not_null][col].mode())
    credit_card[col] = credit_card[col].fillna(mode)
    
sum(credit_card.isnull().sum())

100%|██████████| 9/9 [00:22<00:00,  2.53s/it]


0

### Setup Categorical column

In [22]:
temp = credit_card[["SK_ID_PREV", "NAME_CONTRACT_STATUS"]]

temp = get_dummies(temp, ["NAME_CONTRACT_STATUS"])
temp = temp.drop("NAME_CONTRACT_STATUS", axis=1)
temp = temp.groupby("SK_ID_PREV").sum()

### Select columns

In [23]:
merge_df = pd.DataFrame({
    "AMT_BALANCE": credit_card.groupby("SK_ID_PREV").AMT_BALANCE.mean(),
    "SK_DPD":      credit_card.groupby("SK_ID_PREV").SK_DPD.max(),
    "SK_DPD_DEF":  credit_card.groupby("SK_ID_PREV").SK_DPD_DEF.max(),
    "N":           credit_card.groupby("SK_ID_PREV").count().iloc[:,0]
})

merge_df = temp.join(merge_df)
del temp

### Add the rest of the columns

In [24]:
right = credit_card.drop("SK_ID_CURR", axis=1).groupby("SK_ID_PREV").median().reset_index()
merge_df = merge_df.reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_PREV").set_index("SK_ID_PREV")

<a id="merge_credit"></a>

### [^](#toc) <u>Merge into Previous Application</u>

In [25]:
# Merge
merged_cols = ['credit_' + col for col in merge_df.columns]
merge_df.columns = merged_cols
prev_app = prev_app.merge(right=merge_df.reset_index(), how='left', on='SK_ID_PREV')

### Fill in new NaN values

In [26]:
prev_app["no_credit"] = prev_app[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null = prev_app[col].notnull()
    median = prev_app[not_null][col].median()
    prev_app[col] = prev_app[col].fillna(median)    
    
sum(prev_app.isnull().sum())

100%|██████████| 29/29 [00:21<00:00,  1.32it/s]


19985355

---
<a id="clean_up"></a>

# [^](#toc) <u>Misc clean up</u>

### Drop identification columns

Maybe I shouldn't?  Not all the information may be passed

In [28]:
### Drop unneeded SK_ID_PREV from prev_app
prev_app = prev_app.drop("SK_ID_PREV", axis=1)
# bureau   = bureau.drop("SK_ID_BUREAU", axis=1)

print("Number of null in prev_app:", sum(prev_app.isnull().sum()))
# print("Number of null in bureau:  ", sum(bureau.isnull().sum()))

ValueError: labels ['SK_ID_PREV'] not contained in axis

### Categorical columns

In [30]:
num_cols = [col for col in prev_app.columns if prev_app[col].dtype != object]
cat_cols = [col for col in prev_app.columns if prev_app[col].dtype == object]

print("\nCategoric columns:")
print(" --- ".join(cat_cols))


Numeric columns:
SK_ID_CURR --- AMT_ANNUITY --- AMT_APPLICATION --- AMT_CREDIT --- AMT_DOWN_PAYMENT --- AMT_GOODS_PRICE --- HOUR_APPR_PROCESS_START --- NFLAG_LAST_APPL_IN_DAY --- RATE_DOWN_PAYMENT --- RATE_INTEREST_PRIMARY --- RATE_INTEREST_PRIVILEGED --- DAYS_DECISION --- SELLERPLACE_AREA --- CNT_PAYMENT --- DAYS_FIRST_DRAWING --- DAYS_FIRST_DUE --- DAYS_LAST_DUE_1ST_VERSION --- DAYS_LAST_DUE --- DAYS_TERMINATION --- NFLAG_INSURED_ON_APPROVAL --- DAYS_TERMINATION_maxed --- DAYS_LAST_DUE_maxed --- DAYS_LAST_DUE_1ST_VERSION_maxed --- DAYS_FIRST_DUE_maxed --- DAYS_FIRST_DRAWING_maxed --- AMT_ANNUITY_nan --- AMT_GOODS_PRICE_nan --- CNT_PAYMENT_nan --- AMT_DOWN_PAYMENT_nan --- RATE_DOWN_PAYMENT_nan --- DAYS_FIRST_DRAWING_nan --- DAYS_FIRST_DUE_nan --- DAYS_LAST_DUE_1ST_VERSION_nan --- DAYS_LAST_DUE_nan --- DAYS_TERMINATION_nan --- NFLAG_INSURED_ON_APPROVAL_nan --- pos_NAME_CONTRACT_STATUS_Active --- pos_NAME_CONTRACT_STATUS_Amortized debt --- pos_NAME_CONTRACT_STATUS_Approved --- pos_NAME

### Lump together values with low counts

In [37]:
# NAME_GOODS_CATEGORY
prev_app.NAME_GOODS_CATEGORY = prev_app.NAME_GOODS_CATEGORY.map(
    lambda x: "MISC" if x in ["Weapon", "Insurance"] else x)

# NAME_CASH_LOAN_PURPOSE
prev_app.NAME_CASH_LOAN_PURPOSE = prev_app.NAME_CASH_LOAN_PURPOSE.map(
    lambda x: "MISC" if x in ["Buying a garage", "Misc"] else x)

### Get dummies

FIXME: get rid of this and treat the categorical columns separetly?  Actually maybe this is needed because I just want to sum the counts of each value

In [32]:
bureau   = pd.get_dummies(bureau)

---
<a id="final_merge"></a>

# [^](#toc) <u>Final Data Prep</u>

In [33]:
train = pd.read_csv(DATA_PATH + "train.csv")
test  = pd.read_csv(DATA_PATH + "test.csv")

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 122)
Shape of test: (48744, 121)


### Split into predictors, target, and id

In [34]:
train_y = train.TARGET
train_x = train.drop(["TARGET"], axis=1)

test_id = test.SK_ID_CURR
test_x  = test

### Merge train and test data

In [35]:
full    = pd.concat([train_x, test_x])
train_N = len(train_x)

<a id="final_nan"></a>

### [^](#toc) <u>Missing values</u>

In [36]:
full = fillna_cat(full)
full = fillna_num(full)

sum(full.isnull().sum())

0

### Factorize

In [37]:
# Get categorical features
data_cats = [col for col in full.columns if full[col].dtype == 'object']

# Factorize the dataframe
full = factorize_df(full, data_cats)

### Merge Previous Application with full

In [43]:
cat_cols = [
        "NAME_CONTRACT_TYPE", "WEEKDAY_APPR_PROCESS_START",
        "FLAG_LAST_APPL_PER_CONTRACT", "NAME_CASH_LOAN_PURPOSE",
        "NAME_CONTRACT_STATUS", "NAME_PAYMENT_TYPE",
        "CODE_REJECT_REASON", "NAME_TYPE_SUITE", "NAME_CLIENT_TYPE",
        "NAME_GOODS_CATEGORY", "NAME_PORTFOLIO", "NAME_PRODUCT_TYPE",
        "CHANNEL_TYPE", "NAME_SELLER_INDUSTRY", "NAME_YIELD_GROUP",
        "PRODUCT_COMBINATION", "SK_ID_CURR"]
num_cols = [col for col in prev_app.columns if col not in cat_cols]
num_cols.append("SK_ID_CURR")

merge_df      = prev_app[num_cols].groupby('SK_ID_CURR').mean()
merge_df["N"] = prev_app.groupby('SK_ID_CURR').count().iloc[:,0]

right = prev_app[cat_cols].set_index("SK_ID_CURR")
right = pd.get_dummies(merge_df).reset_index()
right = merge_df.groupby("SK_ID_CURR").sum().reset_index()

merge_df = merge_df.reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_CURR").set_index("SK_ID_CURR")

merged_cols   = ['p_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

full = full.merge(right=merge_df.reset_index(), how='left', on='SK_ID_CURR')

NameError: name 'full' is not defined

#### Fill NaN values

In [39]:
full["no_prev_app"] = full[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null  = full[col].notnull()
    median    = full[not_null][col].median()
    full[col] = full[col].fillna(median)    
    
sum(full.isnull().sum())

100%|██████████| 236/236 [03:52<00:00,  1.02it/s]


0

### Merge Bureau with full

In [40]:
# Average Values for all bureau features 
merge_df         = bureau.groupby('SK_ID_CURR').mean().sort_index()
merge_df['N']    = bureau.groupby('SK_ID_CURR').count().sort_index().iloc[:,0]

### Add the debt to overdue ratio
right = (bureau.groupby("SK_ID_CURR")['AMT_CREDIT_SUM_DEBT'].sum() /
         bureau.groupby("SK_ID_CURR")['AMT_CREDIT_SUM_OVERDUE'].sum() ).sort_index()
merge_df["debt_to_overdue"] = right

### Add the debt to overdue ratio
right = (bureau.groupby("SK_ID_CURR")['AMT_CREDIT_SUM_DEBT'].sum() /
         bureau.groupby("SK_ID_CURR")['AMT_CREDIT_SUM'].sum() ).sort_index()
merge_df["debt_to_credit"] = right

merged_cols = ['b_' + f_ for f_ in merge_df.columns]
merge_df.columns = merged_cols

full = full.merge(right=merge_df.reset_index(), how='left', on='SK_ID_CURR')

#### Fill NaN values

In [41]:
full["no_bureau"] = full[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null  = full[col].notnull()
    median    = full[not_null][col].median()
    full[col] = full[col].fillna(median)    

sum(full.isnull().sum())

100%|██████████| 55/55 [00:53<00:00,  1.03it/s]


0

### Drop SK_ID_CURR

In [43]:
full = full.drop("SK_ID_CURR", axis=1)

### Split full back into train and test

In [44]:
train_x = full[:train_N]
test_x = full[train_N:]

### Processed data look

In [45]:
train_x.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,b_CREDIT_TYPE_Loan for working capital replenishment,b_CREDIT_TYPE_Microloan,b_CREDIT_TYPE_Mobile operator loan,b_CREDIT_TYPE_Mortgage,b_CREDIT_TYPE_Real estate loan,b_CREDIT_TYPE_Unknown type of loan,b_N,b_debt_to_overdue,b_debt_to_credit,no_bureau
0,0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,inf,0.284122,0
1,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,inf,0.000000,0
2,1,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,inf,0.000000,0
3,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,inf,0.210889,1
4,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,inf,0.000000,0


<a id="models"></a>

# [^](#toc) <u>Models </u>

### sban's method

In [48]:
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=17)
lgb_train = lgb.Dataset(data=train_x, label=train_y)
lgb_eval  = lgb.Dataset(data=val_x, label=val_y)

params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 5000, 'verbose': 0 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
          'min_split_gain':.01, 'min_child_weight':1}

start = time.time()
model = lgb.train(params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=150, verbose_eval=200)
print("Training took {} seconds".format(round(time.time() - start)))

Training until validation scores don't improve for 150 rounds.
[200]	valid_0's auc: 0.740448
[400]	valid_0's auc: 0.756613
[600]	valid_0's auc: 0.767844
[800]	valid_0's auc: 0.773388
[1000]	valid_0's auc: 0.776242
[1200]	valid_0's auc: 0.777977
[1400]	valid_0's auc: 0.778871
[1600]	valid_0's auc: 0.779336
[1800]	valid_0's auc: 0.779707
[2000]	valid_0's auc: 0.779993
[2200]	valid_0's auc: 0.780144
[2400]	valid_0's auc: 0.780323
[2600]	valid_0's auc: 0.780392
[2800]	valid_0's auc: 0.780524
[3000]	valid_0's auc: 0.780611
[3200]	valid_0's auc: 0.780652
Early stopping, best iteration is:
[3123]	valid_0's auc: 0.780743
Training took 750 seconds


---
<a id="predictions"></a>

# [^](#toc) <u>Predictions</u>

In [49]:
predictions = model.predict(test_x)

---
<a id="save"></a>

# [^](#toc) <u>Save file to CSV</u>

In [50]:
pd.DataFrame({
    "SK_ID_CURR": test_id,
    "TARGET": predictions
}).to_csv("../submissions/old_method_mapped_max_days.csv", index=False)